## Recommendation Performance

In [12]:
ROOT="/home/sl1471/workspace/experiments/"
best_setting = {'ml-1m': {'MF': ['/logs/f2rec_train_and_eval_MF_lr0.0001_reg0.1_losspairwisebpr.log'], 
                          'FedMF': ['/logs/f2rec_train_and_eval_FedMF_lr0.01_reg0.1_losspairwisebpr_local1_fedavg.log'], 
                          'FairMF': [],
                          'F2MF': []
                         },
                'amz_Books': {'MF': ['/logs/f2rec_train_and_eval_MF_lr0.00001_reg1.0_losspairwisebpr.log'], 
                              'FedMF': [],
                              'FairMF': [],
                              'F2MF': []
                             },
                'amz_Movies_and_TV': {'MF': ['/logs/f2rec_train_and_eval_MF_lr0.0001_reg1.0_losspairwisebpr.log'], 
                                      'FedMF': ['/logs/f2rec_train_and_eval_FedMF_lr0.003_reg0.1_losspairwisebpr_local1_fedavg.log'], 
                                      'FairMF': ['/logs/f2rec_train_and_eval_FairMF_lr0.00001_reg1.0_losspairwisebpr_lambda0.1_gactivity.log'],
                                      'F2MF': []
                                     }
               }

In [13]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import argparse
from argparse import Namespace

def read_line_number(file_path, line_num):
    with open(file_path, 'r') as fin:
        for i,line in enumerate(fin):
            if i == line_num:
                return line.strip()
    return ""

def extract_args(log_path):
    print(log_path)
    argstr = read_line_number(log_path, 1)
    if 'fair_lambda' not in argstr:
        argstr = argstr[:-1] + ', fair_lambda=0.1)'
    if 'fair_rho' not in argstr:
        argstr = argstr[:-1] + ', fair_rho=1)'
    if 'fair_group_feature' not in argstr:
        argstr = argstr[:-1] + f", fair_group_feature='{group_feature}')"
    args = eval(argstr)
    return args

def extract_epochwise_result(log_path):
    '''
    @output:
    - log_result: [round_result]
        - round_result: [epoch_result]
            - epoch_result: {metric: value}
    '''
    log_result, round_result = [], []
    wait_flag, read_flag = False, False
    with open(log_path, 'r') as fin:
        for i,line in enumerate(fin):
            if 'Epoch ' in line:
                wait_flag = True
                if 'Epoch 1' in line:
                    round_result = []
            elif 'Result dict:' in line:
                if wait_flag:
                    read_flag = True
                    wait_flag = False
                else:
                    log_result.append(round_result)
            elif read_flag:
                read_flag = False
                round_result.append(eval(line))
    return log_result
                

In [26]:
group_feature = 'activity'
# data_key = 'ml-1m'
data_key = 'amz_Movies_and_TV'
# data_key = 'amz_Books'

observation_result = extract_epochwise_result(ROOT + data_key + best_setting[data_key]['FedMF'][0])
print(f"#round: {len(observation_result)}")

#round: 1


In [27]:
pick_round = 0
training_curves = {metric: [epoch_result[metric] for epoch_result in observation_result[pick_round]] 
                   for metric in observation_result[pick_round][0]}

In [28]:
training_curves

{'HR@1': [0.30265995686556435,
  0.30300833635375135,
  0.30772058823529413,
  0.3179468242245199,
  0.3111271155923659,
  0.3078034682080925,
  0.3210565476190476,
  0.3069952881478797,
  0.3153252480705623,
  0.3011194029850746,
  0.3142016188373804,
  0.3060485331401666,
  0.2994830132939439,
  0.3066328379847771,
  0.3005524861878453],
 'P@1': [0.30265995686556435,
  0.30300833635375135,
  0.30772058823529413,
  0.3179468242245199,
  0.3111271155923659,
  0.3078034682080925,
  0.3210565476190476,
  0.3069952881478797,
  0.3153252480705623,
  0.3011194029850746,
  0.3142016188373804,
  0.3060485331401666,
  0.2994830132939439,
  0.3066328379847771,
  0.3005524861878453],
 'RECALL@1': [0.049547585261360313,
  0.049273142235839,
  0.05014068350069197,
  0.05204120386812142,
  0.05079180609768871,
  0.05010448434528749,
  0.05231610304378721,
  0.04890330628116911,
  0.05009293950545146,
  0.04869928485870278,
  0.05134629792049993,
  0.05001894058946389,
  0.0487519554772168,
  0.0496